## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import numpy as np 
import os
print(pd.__version__)

1.2.4


## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))



print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [3]:
df.drop(columns='ID').describe()

,Lat,Lng,Distance(mi),Severity
count,6407.000000,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.135189,2.293429
std,0.032555,0.028275,0.396360,0.521225
min,37.609619,-122.510440,0.000000,1.000000
25%,37.737096,-122.412210,0.000000,2.000000
50%,37.768238,-122.404835,0.000000,2.000000
75%,37.787813,-122.392477,0.041000,3.000000
max,37.825626,-122.349734,6.820000,4.000000


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

In [4]:
df['f_Crossing'] , _   = pd.factorize(df['Crossing'])
df['f_Bump'] , _       = pd.factorize(df['Bump'])
df['f_Give_Way'] , _   = pd.factorize(df['Give_Way'])
df['f_Junction'] , _   = pd.factorize(df['Junction'])
df['f_No_Exit'] , _    = pd.factorize(df['No_Exit'])
df['f_Railway'] , _    = pd.factorize(df['Railway'])
df['f_Roundabout'] , _ = pd.factorize(df['Roundabout'])
df['f_Stop'] , _       = pd.factorize(df['Stop'])
df['f_Amenity'] , _    = pd.factorize(df['Amenity'])
df['f_Side'] , _       = pd.factorize(df['Side'])





In [5]:
df.describe()

,ID,Lat,Lng,Distance(mi),Severity,f_Crossing,f_Bump,f_Give_Way,f_Junction,f_No_Exit,f_Railway,f_Roundabout,f_Stop,f_Amenity,f_Side
count,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.00000,6407.0,6407.000000,6407.000000,6407.000000,6407.000000,6407.0,6407.000000,6407.000000,6407.000000
mean,3203.000000,37.765653,-122.405990,0.135189,2.293429,0.08241,0.0,0.000468,0.246449,0.000156,0.026533,0.0,0.097706,0.962853,0.050726
std,1849.685919,0.032555,0.028275,0.396360,0.521225,0.27501,0.0,0.021635,0.430977,0.012493,0.160728,0.0,0.296939,0.189136,0.219454
min,0.000000,37.609619,-122.510440,0.000000,1.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,1601.500000,37.737096,-122.412210,0.000000,2.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000
50%,3203.000000,37.768238,-122.404835,0.000000,2.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000
75%,4804.500000,37.787813,-122.392477,0.041000,3.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000
max,6406.000000,37.825626,-122.349734,6.820000,4.000000,1.00000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [6]:
df=df.drop(columns = ['Crossing','Bump','Give_Way','Junction','No_Exit','Railway','Roundabout','Stop','Amenity','Side'])


In [7]:
df.head()

,ID,Lat,Lng,Distance(mi),Severity,timestamp,f_Crossing,f_Bump,f_Give_Way,f_Junction,f_No_Exit,f_Railway,f_Roundabout,f_Stop,f_Amenity,f_Side
0,0,37.762150,-122.405660,0.044,2,2016-03-25 15:13:02,0,0,0,0,0,0,0,0,0,0
1,1,37.719157,-122.448254,0.000,2,2020-05-05 19:23:00,0,0,0,0,0,0,0,0,1,0
2,2,37.808498,-122.366852,0.000,3,2016-09-16 19:57:16,0,0,0,0,0,0,0,1,1,0
3,3,37.785930,-122.391080,0.009,1,2020-03-29 19:48:43,0,0,0,1,0,0,0,0,1,0
4,4,37.719141,-122.448457,0.000,2,2019-10-09 08:47:00,0,0,0,0,0,0,0,0,1,0


In [8]:
df['time']=pd.to_datetime(df['timestamp'])

df['time_date'] = [d[0:10] for d in df['timestamp']]


df['date']=pd.to_datetime(df['time_date'])

df['hour']=df['time'].dt.hour


In [9]:
df=df.drop(columns = ['timestamp','time','time_date'])


In [10]:
df.head()
df.dtypes

ID                       int64
Lat                    float64
Lng                    float64
Distance(mi)           float64
Severity                 int64
f_Crossing               int64
f_Bump                   int64
f_Give_Way               int64
f_Junction               int64
f_No_Exit                int64
f_Railway                int64
f_Roundabout             int64
f_Stop                   int64
f_Amenity                int64
f_Side                   int64
date            datetime64[ns]
hour                     int64
dtype: object

In [11]:
df.shape


(6407, 17)

In [12]:
df_weather = pd.read_csv( os.path.join(dataset_path, 'weather-sfcsv.csv'))

df_weather["time"]=df_weather[['Year','Month','Day']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

df_weather['date']=pd.to_datetime(df_weather['time'])
df_weather['hour']=df_weather['Hour']

df_weather=df_weather.drop(columns = ['Year','Day','Month',"time",'Hour'])



In [13]:
df_weather.head()

,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,date,hour
0,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No,2020-07-27,18
1,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-09-30,17
2,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-06-27,5
3,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-09-07,9
4,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No,2019-10-19,2


In [14]:
df_weather['Selected'] , _       = pd.factorize(df_weather['Selected'])
df_weather['Weather_Condition'] , _       = pd.factorize(df_weather['Weather_Condition'])

In [15]:
df_weather.head()
df_weather.dtypes

Weather_Condition             int64
Wind_Chill(F)               float64
Precipitation(in)           float64
Temperature(F)              float64
Humidity(%)                 float64
Wind_Speed(mph)             float64
Visibility(mi)              float64
Selected                      int64
date                 datetime64[ns]
hour                          int64
dtype: object

In [16]:
df_weather1=df_weather.drop_duplicates()

df_weather1.shape


(6901, 10)

In [17]:
# df_holiday = pd.read_xml( os.path.join(dataset_path,'holidays.xml'))

# df_holiday['date']=pd.to_datetime(df_holiday['date'])

# df_holiday["holiday"]=1

# print("The shape of the dataset is {}.\n\n".format(df_holiday.shape))                      
# df_holiday.head()

In [18]:
# df_holiday.dtypes

In [19]:
df =pd.merge(df,df_weather1,on=['date','hour'],how='left').drop_duplicates("ID")
# df =pd.merge(df,df_holiday,on='date',how='left')


In [20]:
# df['holiday'] = df['holiday'].fillna(0)
df=df.fillna(df.median())
df.describe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  


,ID,Lat,Lng,Distance(mi),Severity,f_Crossing,f_Bump,f_Give_Way,f_Junction,f_No_Exit,...,f_Side,hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
count,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.00000,6407.0,6407.000000,6407.000000,6407.000000,...,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000
mean,3203.000000,37.765653,-122.405990,0.135189,2.293429,0.08241,0.0,0.000468,0.246449,0.000156,...,0.050726,12.873888,3.417668,59.478726,0.003431,59.938708,68.180115,10.827283,9.440529,0.000312
std,1849.685919,0.032555,0.028275,0.396360,0.521225,0.27501,0.0,0.021635,0.430977,0.012493,...,0.219454,5.824203,3.624300,6.565851,0.021158,7.945247,16.163445,6.350990,1.647528,0.017667
min,0.000000,37.609619,-122.510440,0.000000,1.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,31.100000,0.000000,36.000000,10.000000,0.000000,0.120000,0.000000
25%,1601.500000,37.737096,-122.412210,0.000000,2.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,8.000000,1.000000,59.000000,0.000000,54.000000,59.000000,6.000000,10.000000,0.000000
50%,3203.000000,37.768238,-122.404835,0.000000,2.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,14.000000,3.000000,59.000000,0.000000,59.000000,70.000000,10.000000,10.000000,0.000000
75%,4804.500000,37.787813,-122.392477,0.041000,3.000000,0.00000,0.0,0.000000,0.000000,0.000000,...,0.000000,17.000000,4.000000,59.000000,0.000000,64.900000,80.000000,15.000000,10.000000,0.000000
max,6406.000000,37.825626,-122.349734,6.820000,4.000000,1.00000,0.0,1.000000,1.000000,1.000000,...,1.000000,23.000000,25.000000,98.000000,0.490000,98.000000,100.000000,40.300000,10.000000,1.000000


In [21]:
df=df.drop(columns = ['date'])

In [22]:
corr=df.corr(method ='pearson')
sev=corr['Severity']
sev

ID                   0.020703
Lat                  0.099581
Lng                  0.145313
Distance(mi)        -0.013141
Severity             1.000000
f_Crossing          -0.090314
f_Bump                    NaN
f_Give_Way          -0.012186
f_Junction          -0.068328
f_No_Exit           -0.007034
f_Railway           -0.033322
f_Roundabout              NaN
f_Stop               0.229269
f_Amenity            0.078915
f_Side              -0.060545
hour                -0.019022
Weather_Condition    0.050803
Wind_Chill(F)       -0.014309
Precipitation(in)    0.005668
Temperature(F)      -0.018352
Humidity(%)          0.076773
Wind_Speed(mph)      0.020382
Visibility(mi)      -0.015642
Selected             0.023956
Name: Severity, dtype: float64

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [23]:
from sklearn.model_selection import train_test_split

X_train,X_val ,y_train,y_val = train_test_split(df,df['Severity'], test_size=0.2, stratify=df['Severity']) # Try adding `stratify` here


X_train = X_train.drop(columns=['ID', 'Severity'])

X_val = X_val.drop(columns=['ID', 'Severity'])


In [24]:
# from sklearn.model_selection import train_test_split

# train_df, val_df = train_test_split(df, test_size=0.2, random_state=39) # Try adding `stratify` here

# X_train = train_df.drop(columns=['ID', 'Severity'])
# y_train = train_df['Severity']

# X_val = val_df.drop(columns=['ID', 'Severity'])
# y_val = val_df['Severity']


Weather_Condition	Wind_Chill(F)	Precipitation(in)	Temperature(F)	Humidity(%)	Wind_Speed(mph)	Visibility(mi)	Selected	date	hour

As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [25]:
# # This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.

# X_train = X_train[['Lat', 'Lng', 'Distance(mi)', 'f_Crossing', 'f_Junction', 'f_Give_Way',  'f_Railway','Visibility(mi)', 'f_Stop', 'f_Amenity', 'Selected', 'Weather_Condition','holiday','hour','Wind_Chill(F)','Precipitation(in)','Temperature(F)','Humidity(%)','Wind_Speed(mph)']]
# X_val = X_val    [['Lat', 'Lng', 'Distance(mi)', 'f_Crossing', 'f_Junction', 'f_Give_Way',  'f_Railway','Visibility(mi)', 'f_Stop', 'f_Amenity', 'Selected', 'Weather_Condition','holiday','hour','Wind_Chill(F)','Precipitation(in)','Temperature(F)','Humidity(%)','Wind_Speed(mph)']]

In [26]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.

X_train = X_train[['f_Stop','Lng','Lat','f_Crossing','Humidity(%)','f_Junction','f_Side','Weather_Condition','f_Amenity','Distance(mi)','f_Railway','Selected','Wind_Speed(mph)','Precipitation(in)','f_No_Exit','f_Give_Way','Wind_Chill(F)','hour','f_Side' ]]
X_val = X_val    [['f_Stop','Lng','Lat','f_Crossing','Humidity(%)','f_Junction','f_Side','Weather_Condition','f_Amenity','Distance(mi)','f_Railway','Selected','Wind_Speed(mph)','Precipitation(in)','f_No_Exit','f_Give_Way','Wind_Chill(F)','hour','f_Side']]

Weather_Condition    0.050803
f_Side               0.060545
f_Junction           0.068328
Humidity(%)          0.076773
f_Amenity            0.078915
f_Crossing           0.090314
Lat                  0.099581
Lng                  0.145313
f_Stop               0.229269

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [27]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [28]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7402496099843994


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [29]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))

test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


In [30]:
test_df['f_Crossing'] , _   = pd.factorize(test_df['Crossing'])
test_df['f_Bump'] , _       = pd.factorize(test_df['Bump'])
test_df['f_Give_Way'] , _   = pd.factorize(test_df['Give_Way'])
test_df['f_Junction'] , _   = pd.factorize(test_df['Junction'])
test_df['f_No_Exit'] , _    = pd.factorize(test_df['No_Exit'])
test_df['f_Railway'] , _    = pd.factorize(test_df['Railway'])
test_df['f_Roundabout'] , _ = pd.factorize(test_df['Roundabout'])
test_df['f_Stop'] , _       = pd.factorize(test_df['Stop'])
test_df['f_Amenity'] , _    = pd.factorize(test_df['Amenity'])
test_df['f_Side'] , _       = pd.factorize(test_df['Side'])




In [31]:

test_df['time']=pd.to_datetime(test_df['timestamp'])

test_df['time_date'] = [d[0:10] for d in test_df['timestamp']]


test_df['date']=pd.to_datetime(test_df['time_date'])

test_df['hour']=test_df['time'].dt.hour


In [32]:
test_df =pd.merge(test_df,df_weather1,on=['date','hour'],how='left').drop_duplicates("ID")
# test_df =pd.merge(test_df,df_holiday,on='date',how='left')


In [33]:

# test_df['holiday'] = test_df['holiday'].fillna(0)
test_df=test_df.fillna(df.median())


In [34]:
test_df=test_df.drop(columns = ['Crossing','Bump','Give_Way','Junction','No_Exit','Railway','Roundabout','Stop','Amenity','Side'])

Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [35]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['f_Stop','Lng','Lat','f_Crossing','Humidity(%)','f_Junction','f_Side','Weather_Condition','f_Amenity','Distance(mi)','f_Railway','Selected','Wind_Speed(mph)','Precipitation(in)','f_No_Exit','f_Give_Way','Wind_Chill(F)','hour','f_Side' ]]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [36]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.